In [3]:
### import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Model Parameters
pixel_width = 28
pixel_height = 28
input_shape = (pixel_width, pixel_height, 1)
num_of_classes = 10
batch_size = 32  # Lower batch size to reduce memory usage
epochs = 10  # Reduce epochs to avoid long training times
learning_rate = 0.001  # Base learning rate
dropout_rate = 0.4  # Adjusted dropout rate to prevent overfitting

# Load MNIST Dataset
(features_train, labels_train), (features_test, labels_test) = keras.datasets.mnist.load_data()

# Reshape and Normalize Data
features_train = features_train.reshape(-1, pixel_width, pixel_height, 1).astype("float32") / 255.0
features_test = features_test.reshape(-1, pixel_width, pixel_height, 1).astype("float32") / 255.0

# One-hot encode labels
labels_train = keras.utils.to_categorical(labels_train, num_of_classes)
labels_test = keras.utils.to_categorical(labels_test, num_of_classes)

# 🏆 **Simplified Data Augmentation**
datagen = ImageDataGenerator(
    rotation_range=10,  # Slightly smaller rotation range
    zoom_range=0.1,  # Reduced zoom range
    width_shift_range=0.1,  # Slightly smaller horizontal shift
    height_shift_range=0.1,  # Slightly smaller vertical shift
    shear_range=0.05,  # Reduced shear range
)
datagen.fit(features_train)

# 🧠 **Deep CNN Model with Regularization**
model = Sequential()

# First Conv Block
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_regularizer=l2(0.0005)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(dropout_rate))

# Second Conv Block
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(dropout_rate))

# Fully Connected Layers
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.0005)))
model.add(Dropout(dropout_rate))  # Apply dropout in the FC layer
model.add(Dense(num_of_classes, activation='softmax'))

# **Optimizer with Learning Rate Decay**
optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# 🧠 **Advanced Callbacks**: Patience for early stopping and reduce LR
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7, verbose=1)

# 🚀 **Train Model with Augmented Data**
model.fit(datagen.flow(features_train, labels_train, batch_size=batch_size),
          validation_data=(features_test, labels_test),
          epochs=epochs,
          verbose=1,
          callbacks=[early_stopping, reduce_lr])

# 🎯 **Evaluate Model**
score = model.evaluate(features_test, labels_test, verbose=0)
print(f"🔥 Final Loss: {score[0]:.5f}, Accuracy: {score[1] * 100:.2f}%")

# Save the trained model
model.save('model.h5')  # Save model to a file
print("Model saved as 'model.h5'")


C:\ProgramData\anaconda3\envs\mnist_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\ProgramData\anaconda3\envs\mnist_env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 138s 72ms/step - accuracy: 0.7729 - loss: 1.1073 - val_accuracy: 0.9775 - val_loss: 0.3783 - learning_rate: 0.0010
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 142s 76ms/step - accuracy: 0.9406 - loss: 0.5180 - val_accuracy: 0.9844 - val_loss: 0.3909 - learning_rate: 0.0010
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 152s 81ms/step - accuracy: 0.9517 - loss: 0.5102 - val_accuracy: 0.9886 - val_loss: 0.3671 - learning_rate: 0.0010
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 156s 83ms/step - accuracy: 0.9556 - loss: 0.4844 - val_accuracy: 0.9890 - val_loss: 0.3455 - learning_rate: 0.0010
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 154s 82ms/step - accuracy: 0.9583 - loss: 0.4603 - val_accuracy: 0.9850 - val_loss: 0.3589 - learning_rate: 0.0010
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 154s 82ms/step - accuracy: 0.9622 - loss: 0.4381 - val_accuracy: 0.9875 - val_loss: 0.3174 - learning_rate: 0.0010
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 157s 84ms/step -

🔥 Final Loss: 0.26537, Accuracy: 99.19%
Model saved as 'model.h5'
